In [1]:
# Load data for SST and LWP from ICON data:

# Temporal variability of measurement against spacial variability of model:

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("/home/qwertz/Dokumente/N_eigene_Dokumente/Geo_Met_MSc/Semester_4/Thesis_prep_Juni_23/Porject_Work/")
from Sc_module import convert_single_timestamp

###################################

file1 = "/home/qwertz/data_js_icon_22_23_August_2018/cloud_DOM01.20180822.selvar.int.nc"
file2 = "/home/qwertz/data_js_icon_22_23_August_2018/cloud_DOM01_ML_20180823.selvar.int.nc"
file3 = "/home/qwertz/data_js_icon_22_23_August_2018/cloud_DOM01_ML_20180822_all.sfc.nc"
file4 = "/home/qwertz/data_js_icon_22_23_August_2018/cloud_DOM01_ML_20180823_all.sfc.nc"
gridfile = "/home/qwertz/data_js_icon_22_23_August_2018/chile_square_0628m.nc"

ds_icon22 = xr.open_dataset(file1, engine="netcdf4")
ds_icon23 = xr.open_dataset(file2, engine="netcdf4")
ds_icon_sfc22 = xr.open_dataset(file3, engine="netcdf4")
ds_icon_sfc23 = xr.open_dataset(file4, engine="netcdf4")
gridles = xr.open_dataset(gridfile, engine="netcdf4")
gridglobal = xr.open_dataset("/home/qwertz/forcing_icon/icon_grid_chile_R03B07_20180103.nc")

#######################################
# 
lwp_icon = xr.concat([ds_icon22["tqc"], ds_icon23["tqc"]], dim="time")
# DataArray LWP (time:277, ncells: 252448)
# sfc_icon = xr.concat([ds_icon_sfc22[""], ds_icon_sfc23[""]], dim="time")

print(ds_icon_sfc22.data_vars)

##########################################
# SSTs from ICON: 

path = "/home/qwertz/forcing_icon/icon20180822/entpackt/2018/08/"
files = ["2018082200_icon.nc", "2018082212_icon.nc" , "2018082300_icon.nc",  "2018082312_icon.nc"]

for file in files:
    ds = xr.open_dataset(path+file)

    print(ds.data_vars)
    print(ds["time"])

    print("\n\n")

##########################################



/home/qwertz/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/qwertz/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Data variables:
    clon_bnds    (ncells, vertices) float32 3MB ...
    clat_bnds    (ncells, vertices) float32 3MB ...
    height_bnds  (height, bnds) float64 16B ...
    qv           (time, height, ncells) float32 145MB ...
    qc           (time, height, ncells) float32 145MB ...
    qr           (time, height, ncells) float32 145MB ...
    u            (time, height, ncells) float32 145MB ...
    v            (time, height, ncells) float32 145MB ...
    w            (time, height_3, ncells) float32 145MB ...
    temp         (time, height, ncells) float32 145MB ...
    pres         (time, height, ncells) float32 145MB ...
    theta_v      (time, height, ncells) float32 145MB ...
    z_mc         (height, ncells) float32 1MB ...
Data variables:
    height_bnds    (height, bnds) float64 1kB ...
    height_2_bnds  (height_2, bnds) float64 1kB ...
    HHL            (height, ncells) float32 3MB ...
    T_G            (time, ncells) float32 324kB ...
    sr             (ncells) float32 

In [21]:
# Dependence of PBL(SST)

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
from scipy.interpolate import griddata

# I do not have ICON raw data from the whole domain.
# I could check PBL(SST) depending on one of the slices, that Jan produced:
files = glob.glob("/home/qwertz/data_js_icon_22_23_August_2018/cloud_DOM01_ML_2018082*_all.lat_2141.xz-sect.nc")
ds_icon = xr.open_mfdataset(files)
otherfiles = glob.glob("/home/qwertz/forcing_icon/icon20180822/entpackt/2018/08/2018*.nc")
ds_sst1 = xr.open_mfdataset(otherfiles[0])
ds_sst2 = xr.open_mfdataset(otherfiles[1])
ds_sst3 = xr.open_mfdataset(otherfiles[2])
ds_sst4 = xr.open_mfdataset(otherfiles[3])
gridglobal = xr.open_dataset("/home/qwertz/forcing_icon/icon20180822/icon_grid_chile_R03B07_20180103.nc", engine="netcdf4")


In [24]:
#############
# Problem one: Match data from ICON LES with coarse ICON global SSTs:

# Probably I should not regrid about lats and lons but with lons and height..
# and you have to do that for every timestep...
new_lons = np.linspace(np.nanmin(ds_icon["clon"].values),np.nanmax(ds_icon["clon"].values), 2000)
new_heights = np.linspace(10, 3000, 150) # bis 3000 m ca. 20 m Abstand im vertikalen Grid.
lons_field, heights_field = np.meshgrid(new_lons, new_heights)
old_lons = ds_icon["clon"].values
########################################
original_values =  ds_icon["qc"].values
########################################
new_field = np.zeros([len(ds_icon["time"]), len(new_lons), len(new_heights)])

for i, timestep in enumerate(ds_icon["time"].values):
    old_heights = ds_icon["z_mc"].isel(time=i).values
    new_field[i,:,:] = griddata((old_lons, old_heights), original_values, (lons_field.ravel(), heights_field.ravel()))

###
# Make this block a function that you can adjust for different variables and grids, that you want them to interpolate to...
# Use it for Theta, qc and qv 
# Write results into a netcdf file, when you are sure the process is working and never calc this again!


##################
# Now grid SST on then same lat and lons:
print(gridglobal)
print(ds_sst1["T_G"])



[2.0859828e+04 2.0599156e+04 2.0365080e+04 2.0142426e+04 1.9927648e+04
 1.9718984e+04 1.9515359e+04 1.9316051e+04 1.9120535e+04 1.8928416e+04
 1.8739385e+04 1.8553186e+04 1.8369615e+04 1.8188498e+04 1.8009686e+04
 1.7833049e+04 1.7658475e+04 1.7485865e+04 1.7315133e+04 1.7146197e+04
 1.6978990e+04 1.6813443e+04 1.6649500e+04 1.6487107e+04 1.6326214e+04
 1.6166774e+04 1.6008747e+04 1.5852092e+04 1.5696771e+04 1.5542754e+04
 1.5390004e+04 1.5238494e+04 1.5088193e+04 1.4939077e+04 1.4791118e+04
 1.4644293e+04 1.4498579e+04 1.4353954e+04 1.4210398e+04 1.4067891e+04
 1.3926413e+04 1.3785948e+04 1.3646479e+04 1.3507987e+04 1.3370458e+04
 1.3233877e+04 1.3098229e+04 1.2963499e+04 1.2829676e+04 1.2696745e+04
 1.2564694e+04 1.2433512e+04 1.2303187e+04 1.2173706e+04 1.2045061e+04
 1.1917238e+04 1.1790230e+04 1.1664027e+04 1.1538618e+04 1.1413995e+04
 1.1290148e+04 1.1167070e+04 1.1044751e+04 1.0923184e+04 1.0802360e+04
 1.0682272e+04 1.0562914e+04 1.0444276e+04 1.0326353e+04 1.0209137e+04
 1.009

In [ ]:
############
# Problem two: There are a million ways to find PBL (cloudtop qc or Ri number or those from UCLA manual)
# Probably a method depending on gradients in Theta and q is the best!


In [ ]:
# Calc here: Dependence LWP(SST)

In [3]:
# Similar observation data? (at least SST)

In [ ]:
# Other parameter?

#######################################################

# I would need SSTs from ICON
# I also lack a highly resolved dataset that is not from ICON.